# Covid-19 Dashboard

In [20]:
# Import necessary libraries
import ipywidgets as wdg
import matplotlib.pyplot as plt
import pickle
import time
from uk_covid19 import Cov19API
import timeseries

# Include graphs in notebook
%matplotlib inline
# Make graphs larger
plt.rcParams['figure.dpi'] = 100

# Define variables for the timeseries object
file_name = 'timeseriesdf.pkl'
columns = ['cases', 'hospital', 'deaths']
filters = ['areaType=overview']
structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate"
        }
# Create timeseries object
timeseries = timeseries.timeseries(file_name = file_name, columns = columns, filters = filters, structure = structure)

# Function to trigger access_api function when the refresh button is clicked 
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Try calling the api
    try:
        timeseries.access_api()
        # Format the data for plotting
        timeseries.wrangle_data()
        # Save the outputs to the pickle file for when the dashboard is next viewd
        timeseries.df.to_pickle(file_name)
        # Trigger the graph to refresh
        refresh_graph()
        # Update the Button to show the API was successful
        apibutton.description='Successful!'
        apibutton.icon="check"
        apibutton.button_style='success'
        apibutton.disabled=True
        time.sleep(3)
        # Reset the Button 
        apibutton.description='Refresh Data'
        apibutton.icon="refresh"
        apibutton.button_style='primary'
        apibutton.disabled=False
    except:
        # Update the Button to show the API failed
        apibutton.description='Error'
        apibutton.icon="fa-exclamation-triangle"
        apibutton.button_style='warning'
        apibutton.disabled=True
        time.sleep(3)
        # Reset the Button
        apibutton.description='Refresh Data'
        apibutton.icon="refresh"
        apibutton.button_style='primary'
        apibutton.disabled=False
    
apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='primary', 
    tooltip="Click to refresh the data in the graph",
    icon='refresh'
)

# Register button callback function with the button
apibutton.on_click(api_button_callback)

## Time Series Graph of Cases, Hospitalisations and Deaths

Cases relates to daily case relates

Hospital relates to new hospital admissions

Deaths relates to cumulative deaths within 28 days of positive test by death date per 100k resident population

Please refresh the data for the latest view of the graph

In [21]:
# Create a function to flick between log scale and columns displayed on the graph
def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseries.df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# Create a widget for the user to select columns
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

# Create a button to allow users to flick between log and linear displays
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back"""

    
controls=wdg.VBox([series, scale, apibutton])
    
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

form=wdg.HBox([graph, controls])    
display(form)

**Author and Copyright Notice** (C) Guy Hunt, 2020 (g.p.w.hunt@se20.qmul.ac.uk). All rights reserved.: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*